# Bllossom-AICA Instruction Tuning Tutorial (Only Text)

## Library Import

In [ ]:
!pip install datasets bitsandbytes peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from transformers import MllamaForConditionalGeneration, AutoTokenizer
from transformers import Trainer, TrainingArguments
import torch
from torch.nn.utils.rnn import pad_sequence
import datasets
from peft import LoraConfig, get_peft_model

os.environ['CUDA_VISIBLE_DEVICES']="0"

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `zzzzzzzzzzzzzzzzzzzz` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `zzz

## Model & Tokenizer Load

In [ ]:
model_id = 'Bllossom/llama-3.2-Korean-Bllossom-AICA-5B'

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map='cuda:0',
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/5.22k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/84.7k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/835M [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [ ]:
# 학습 전
sample_message = [
    {'role':'user', 'content': [
        {'type':'text', 'text':'자연어처리 교과목에 대해 간략히 소개해줘'}
        ]},
]

inputs = tokenizer.apply_chat_template(sample_message,
                              tokenize=True,
                              add_generation_prompt=True,
                              return_tensors='pt').to(model.device)


output = model.generate(inputs,max_new_tokens=512,eos_token_id=tokenizer.convert_tokens_to_ids('<|eot_id|>'))

print(tokenizer.decode(output[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 04 Apr 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

자연어처리 교과목에 대해 간략히 소개해줘<|eot_id|><|start_header_id|>assistant<|end_header_id|>

자연어처리(Natural Language Processing, NLP)는 컴퓨터가 인간의 언어를 이해하고 생성할 수 있도록 하는 기술을 말합니다. 이는 컴퓨터 과학, 인공지능, 언어학, 심리학 등 다양한 분야의 지식을 통합하여 발전해 있습니다. 자연어처리는 주요로 다음과 같은 분야와 문제들에게 적용됩니다:

1. **언어 이해**: 텍스트나 음조 정보를 바탕으로 그 내용을 이해하고, 의미를 추출하는 기술입니다. 여기서 문맥을 이해하고, 명령어를 인식하며, 회화 형태의 질문에 답변하는 등의 작업이 포함됩니다.

2. **언어 생성**: 컴퓨터가 인간이 사용하는 언어의 형태로 문장을, 도움말, 이야기 등을 생성할 수 있는 기술입니다. 예를 들어, 자연어로답니다, 즉, 궁금한 점이나 문제가 있다면 설명해 줄 수 있다가며, 문학 작품이나 독창적인 글을 작성할 수도 있습니다.

3. **언어 문법 및 구조 분석**: 텍스트의 문법 구조를 분석하여 의미 부호, 문장 구조, 단어의 의미 등을 파악하는 작업입니다. 이를 통해 텍스트의 의미를 정확하게 해석할 수 있습니다.

4. **음성 인식 및 합성**: 음성Command를 음성 파일로 변환하거나, 텍스트를 음성서식으로 변환하는 기술입니다. 이는 음성 브라우저나 가상 보조기를 만들 수 있는 기초가 됩니다.

5. **번역**: 하나의 언어에서 다른 언어로 텍스트의 의미를 전달하는 작업입니다. 이는 기계 번역으로, 예를 들어 영어를 한국어로 번역하는 기능입니다.

자연어처리는 

## Dataset Load

In [ ]:
train_dataset = datasets.load_dataset('beomi/KoAlpaca-v1.1a',split='train[:100]')
train_dataset

README.md:   0%|          | 0.00/1.75k [00:00<?, ?B/s]

(…)-00000-of-00001-21df739eb88d711e.parquet:   0%|          | 0.00/12.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21155 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'output', 'url'],
    num_rows: 100
})

In [ ]:
train_dataset[0]

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}

## Data Preprocessing

In [ ]:
def preprocessing_data(examples):
    input_ids = []
    attention_masks = []
    labels = []

    for instruction,response in zip(examples['instruction'],examples['output']):
        message = [
        {'role': 'user', 'content': [
            {'type':'text', 'text':instruction}
        ]},
        ]
        inputs = tokenizer.apply_chat_template(message, tokenize=True, add_generation_prompt=True)
        label = tokenizer(response+'<|eot_id|>',add_special_tokens=False)['input_ids']
        input_id = inputs+label+([tokenizer.pad_token_id]*4096)
        input_id = input_id[:4096]
        label_id = [-100]*len(inputs) + label + ([tokenizer.pad_token_id]*4096)
        label_id = label_id[:4096]
        attention_mask = [1 if token != tokenizer.pad_token_id else 0 for token in input_id]

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        labels.append(label_id)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_masks,
        'labels': labels
    }

In [ ]:
train_dataset = train_dataset.map(
    preprocessing_data,
    num_proc=2,
    batched=True,
    remove_columns=['instruction','output','url']
)

train_dataset

Map (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})

In [ ]:
train_dataset[0]["input_ids"][:10]

[128000, 128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25]

In [ ]:
tokenizer.decode(train_dataset[0]["input_ids"][:100])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 04 Apr 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해'

## Data Collator

In [ ]:
def DataCollator(examples):
    input_ids = torch.LongTensor([example['input_ids'] for example in examples])
    attention_mask = torch.LongTensor([example['attention_mask'] for example in examples])
    labels = torch.LongTensor([example['labels'] for example in examples])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }


## Configuration TrainingArugments

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    remove_unused_columns=False,
    report_to="none",
    optim="adamw_bnb_8bit",
    gradient_checkpointing=True,
    num_train_epochs=1,
    logging_strategy='steps',
    logging_steps=10,
    label_names=['labels'],
    torch_compile=True,
)


## Configuration Trainable Parameters

In [ ]:
# LLM Full Tuning
target_modules = []
for n,p in model.named_parameters():
    if 'language_model' in n and 'cross_attn' not in n and 'embed_tokens' not in n and 'lm_head' not in n and 'norm' not in n:
        target_modules.append(n.replace('.weight',''))
    # else:
    #     p.require_grad=False

# Lora Tuning
peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=4,  #<-- 요거 줄이면 GPU메모리 절약됩니다.
    lora_alpha=8, #<-- 요거 줄이면 GPU메모리 절약됩니다.
    target_modules=target_modules
)

lora_model = get_peft_model(model,peft_config)
lora_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MllamaForConditionalGeneration(
      (vision_model): MllamaVisionModel(
        (patch_embedding): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), padding=valid, bias=False)
        (gated_positional_embedding): MllamaPrecomputedPositionEmbedding(
          (tile_embedding): Embedding(9, 8197120)
        )
        (pre_tile_positional_embedding): MllamaPrecomputedAspectRatioEmbedding(
          (embedding): Embedding(9, 5120)
        )
        (post_tile_positional_embedding): MllamaPrecomputedAspectRatioEmbedding(
          (embedding): Embedding(9, 5120)
        )
        (layernorm_pre): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (layernorm_post): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (transformer): MllamaVisionEncoder(
          (layers): ModuleList(
            (0-31): 32 x MllamaVisionEncoderLayer(
              (self_attn): MllamaVisionSdpaAttention(
               

## Train

In [ ]:
# LLM 풀튜닝 VRAM 30GB 사용
# LoRA 사용시 18.6GB 사용됨 (RANK 16, 토큰임베딩 및 lm_head 학습 X)
# 메모리 터질 시 RANK 사이즈 등 조정해서 사용하세용~
trainer = Trainer(
        model=lora_model,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=DataCollator,
    )

trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,10.911700
20,8.064000
30,6.624000
40,6.065800
50,5.791100
60,5.717900
70,5.612500
80,5.626200
90,5.649000
100,5.623600


TrainOutput(global_step=100, training_loss=6.568589935302734, metrics={'train_runtime': 416.0243, 'train_samples_per_second': 0.24, 'train_steps_per_second': 0.24, 'total_flos': 1.1678062374912e+16, 'train_loss': 6.568589935302734, 'epoch': 1.0})

## Inference Test

In [ ]:
# 학습 후
sample_message = [
    {'role':'user', 'content': [
        {'type':'text', 'text':'서울의 유명 관광지에 대해 소개해줘'}
        ]},
]

inputs = tokenizer.apply_chat_template(sample_message,
                              tokenize=True,
                              add_generation_prompt=True,
                              return_tensors='pt').to(model.device)


output = lora_model.generate(inputs,max_new_tokens=512,eos_token_id=tokenizer.convert_tokens_to_ids('<|eot_id|>'))

print(tokenizer.decode(output[0]))

/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 04 Apr 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

서울의 유명 관광지에 대해 소개해줘<|eot_id|><|start_header_id|>assistant<|end_header_id|>

서울은 동아시아에서 가장 큰 도시 중 하나로, 역사를 이어받고 있는 역사적 건물과 현대적인 건축물이 공존하는 도시입니다. 서울의 유명 관광지는 크게 역사적인 장소와 건축물, 그리고 현대적인 명소들로 나눌 수 있습니다. 여기서는 몇 가지 주요 관광지를 소개하겠습니다.

### 역사와 문화
- **경복궁**: 한국의 대표적인 궁궐 중 하나로, 광화원 왕원실을 중심으로 마련되어 있습니다. 경복궁은 1395년에 건립되었으며, 그 역사와 아름다움을 느끼기에 충분한 시간을 할애해야 합니다.
- **청경댄**: 한강의 대류 지점인 청경댈로는, 연못 경관과 함께 1900년대 초에 건립된 시설을 감상할 수 있는 곳입니다.
- **남산타워**: 서울에서 한강을 내려다볼 수 있는 최고의 전망대 중 하나입니다. 남산 정상과 함께 서울의 아름다운 전경을 감상할 수 있습니다.

### 현대적인 명소
- **남대문시장**: 세계적으로 유명한 시크림과 같은 다양한 상품을 볼 수 있는 시장입니다. 또한 다양한 전시와 공연을 쉽게 즐길 수 있어 방문객들에게 큰 인기를 끌고 있습니다.
- **북촌 한옥마을**: 서울에서 가장 유명한 전통 한옥 거리 중 하나로, 가슴진 고풍스러운 한옥과 함께 전통 가옥과 한옥을 볼 수 있는 곳입니다. 한국과 일본 관광객들에게 매우 인기가 높습니다.
- **서울타워**: 한국 유시티, 그리고 두 대의 하나와 같은 개선과 같은 뚜려헜은 높은 아름다운 스컬피니다. 두 개, 하나와 같이 해대는 마중을 세우다가 있습니다.

##